In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree, export_text
# import math
from graphviz import Digraph
import math


In [2]:

# col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
# data = pd.read_csv(r"./Iris.csv", skiprows=1, header=None, names=col_names)

col_names = ['Holiday', 'Discount', 'Free Delivery', 'Purchase']
data = pd.read_csv(r"./Purchase_new.csv", skiprows=1, header=None, names=col_names)
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.dot = Digraph(comment='Decision Tree')

    # def build_tree(self, dataset, curr_depth=0):
    #     X, Y = np.array([row[:-1] for row in dataset]), np.array([row[-1] for row in dataset])
    #     num_samples, num_features = self.shape(X)

    #     if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
    #         best_split = self.get_best_split(dataset, num_samples, num_features)
    #         if best_split["info_gain"] > 0:
    #             left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
    #             right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
    #             return Node(best_split["feature_index"], best_split["threshold"],
    #                         left_subtree, right_subtree, best_split["info_gain"])

    #     leaf_value = self.calculate_leaf_value(Y)
    #     return Node(value=leaf_value)
    
    def build_tree(self, dataset, curr_depth=0):
        X, Y = np.array([row[:-1] for row in dataset]), np.array([row[-1] for row in dataset])
        num_samples, num_features = self.shape(X)

        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split and "info_gain" in best_split and best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split.get("info_gain", None))

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)


    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")

        for feature_index in range(num_features):
            feature_values = np.array([row[feature_index] for row in dataset])
            possible_thresholds = self.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = np.array([row[-1] for row in dataset]), np.array([row[-1] for row in dataset_left]), np.array([row[-1] for row in dataset_right])
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        return best_split

    def split(self, dataset, feature_index, threshold):
        dataset_left = [row for row in dataset if row[feature_index] <= threshold]
        dataset_right = [row for row in dataset if row[feature_index] > threshold]
        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
        return gain

    def entropy(self, y):
        class_labels = self.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len([label for label in y if label == cls]) / len(y)
            entropy += -p_cls * self.log2(p_cls)
        return entropy

    def gini_index(self, y):
        class_labels = self.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len([label for label in y if label == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini

    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)

    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_" + str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        dataset = X.tolist()
        for i, y in enumerate(Y):
            dataset[i].append(y)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

    def make_prediction(self, x, tree):
        if tree.value is not None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

    def shape(self, array):
        return len(array), len(array[0])

    def unique(self, array):
        return np.unique(array)

    def log2(self, x):
        return 0 if x == 0 else math.log2(x)

    def visualize_tree(self, tree=None):
        if not tree:
            tree = self.root

        if tree.value is not None:
            self.dot.node(str(id(tree)), str(tree.value), shape='oval', style='filled', color='lightblue')

        else:
            self.dot.node(str(id(tree)), f'X_{str(tree.feature_index)} <= {str(tree.threshold)}\nInfo Gain: {str(tree.info_gain)}', shape='box', style='filled', color='lightgreen')
            if tree.left:
                self.dot.edge(str(id(tree)), str(id(tree.left)), label='True')
                self.visualize_tree(tree.left)
            if tree.right:
                self.dot.edge(str(id(tree)), str(id(tree.right)), label='False')
                self.visualize_tree(tree.right)

    def save_tree_graph(self):
        self.visualize_tree()
        self.dot.render('decision_tree', view=True)

# Split the data and fit the model
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1, 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train, Y_train)
classifier.save_tree_graph()

In [3]:
#     # Split the data and fit the model
# X = df.iloc[:, :-1].values
# Y = df.iloc[:, -1].values.reshape(-1, 1)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

# classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
# classifier.fit(X_train, Y_train)
# classifier.save_tree_graph()